In [1]:
# 1. imports
import pandas as pd
import numpy as np
import datetime
from pathlib import Path

# for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# optional: display settings
pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
# 2. scenario definitions

# Time: quarterly periods over several years (example 2019–2024)
quarters = pd.period_range('2019Q1', '2024Q4', freq='Q').astype(str).tolist()

# Geography: USA states where Van Leeuwen operates, plus a few international countries.
# This mirrors nationwide presence; you can expand or trim:
us_states = [
    'NY', 'CA', 'NJ', 'TX', 'PA', 'CO', 'CT', 'DC',
    'MA', 'FL', 'TN', 'GA', 'NC', 'IL'
]
international_countries = ['Canada', 'UK', 'Australia', 'Japan', 'Germany']

# Channel types
channels = ['Scoop Shop', 'Retail Grocery', 'Online Delivery', 'Wholesale']

# Product lines (examples)
product_lines = [
    'Classic Ice Cream', 'Vegan Ice Cream', 'Ice Cream Bars',
    'Ice Cream Sandwiches', 'Seasonal Specials'
]

# Demographics buckets
age_groups = ['<18', '18-34', '35-54', '55+']
income_levels = ['Low', 'Medium', 'High']
household_sizes = ['1', '2', '3-4', '5+']
genders = ['Male', 'Female', 'Non-binary/Other']

In [3]:
# 3. store list generation

stores = []
store_id = 1000

# US stores
for state in us_states:
    store_id += 1
    stores.append({
        'store_id': store_id,
        'store_name': f'Van Leeuwen {state} Store',
        'country': 'USA',
        'state_or_region': state,
        'city': f'City_{state}',
        'opening_year': np.random.randint(2019, 2023)
    })

# International stores
for country in international_countries:
    store_id += 1
    stores.append({
        'store_id': store_id,
        'store_name': f'Van Leeuwen {country} Store',
        'country': country,
        'state_or_region': None,
        'city': f'City_{country}',
        'opening_year': np.random.randint(2020, 2024)
    })

stores_df = pd.DataFrame(stores)
stores_df.head()

,store_id,store_name,country,state_or_region,city,opening_year
0,1001,Van Leeuwen NY Store,USA,NY,City_NY,2021
1,1002,Van Leeuwen CA Store,USA,CA,City_CA,2022
2,1003,Van Leeuwen NJ Store,USA,NJ,City_NJ,2019
3,1004,Van Leeuwen TX Store,USA,TX,City_TX,2021
4,1005,Van Leeuwen PA Store,USA,PA,City_PA,2021


In [4]:
# 4. master grid
rows = []
for quarter in quarters:
    for _, store in stores_df.iterrows():
        # possible channels per store – could vary; here assume all channels exist
        for channel in channels:
            for pl in product_lines:
                rows.append({
                    'quarter': quarter,
                    'store_id': store['store_id'],
                    'channel': channel,
                    'product_line': pl
                })

master_df = pd.DataFrame(rows)
master_df.shape

(9120, 4)

In [5]:
# 5. synthetic financials

def generate_sales(row):
    # base sales by channel
    base = {
        'Scoop Shop': 200_000,
        'Retail Grocery': 150_000,
        'Online Delivery': 80_000,
        'Wholesale': 120_000
    }[row['channel']]
    
    # adjust for international vs USA
    if row['store_id'] > 2000:  # arbitrary split for international stores from earlier generation
        base *= 0.6  # smaller initial base, could adjust as desired
    
    # random variation
    noise = np.random.normal(loc=1.0, scale=0.2)  # 20% variation
    return max(0, base * noise)

def generate_profit_margin(row):
    # base margin per product line
    base_margin = {
        'Classic Ice Cream': 0.30,
        'Vegan Ice Cream': 0.28,
        'Ice Cream Bars': 0.25,
        'Ice Cream Sandwiches': 0.27,
        'Seasonal Specials': 0.32
    }[row['product_line']]
    
    # channel effect
    channel_adj = {
        'Scoop Shop': 0.0,
        'Retail Grocery': -0.03,
        'Online Delivery': -0.05,
        'Wholesale': -0.07
    }[row['channel']]
    
    noise = np.random.normal(loc=0, scale=0.02)  # small random noise
    margin = base_margin + channel_adj + noise
    return max(0, min(margin, 0.5))  # clamp to sensible [0, .5]

master_df['sales'] = master_df.apply(generate_sales, axis=1)
master_df['profit_margin'] = master_df.apply(generate_profit_margin, axis=1)

# optionally round
master_df['sales'] = master_df['sales'].round(2)
master_df['profit_margin'] = master_df['profit_margin'].round(4)

master_df.head()

,quarter,store_id,channel,product_line,sales,profit_margin
0,2019Q1,1001,Scoop Shop,Classic Ice Cream,"180,695.25",0.29
1,2019Q1,1001,Scoop Shop,Vegan Ice Cream,"206,566.59",0.27
2,2019Q1,1001,Scoop Shop,Ice Cream Bars,"209,323.81",0.23
3,2019Q1,1001,Scoop Shop,Ice Cream Sandwiches,"204,719.78",0.29
4,2019Q1,1001,Scoop Shop,Seasonal Specials,"258,495.12",0.31


In [6]:
# 6. demographic distributions

def random_distribution(categories):
    # sample random weights, normalize to sum 1
    vals = np.random.rand(len(categories))
    vals /= vals.sum()
    return vals

# generate demographic share columns
age_dist = np.array([random_distribution(age_groups) for _ in range(len(master_df))])
income_dist = np.array([random_distribution(income_levels) for _ in range(len(master_df))])
hh_dist = np.array([random_distribution(household_sizes) for _ in range(len(master_df))])
gender_dist = np.array([random_distribution(genders) for _ in range(len(master_df))])

# assign column names and add to df
for i, ag in enumerate(age_groups):
    master_df[f'age_{ag}'] = age_dist[:, i]
for i, inc in enumerate(income_levels):
    master_df[f'income_{inc}'] = income_dist[:, i]
for i, hh in enumerate(household_sizes):
    master_df[f'household_{hh}'] = hh_dist[:, i]
for i, g in enumerate(genders):
    safe_col = g.replace(' ', '_').replace('/', '_').replace('-', '_')
    master_df[f'gender_{safe_col}'] = gender_dist[:, i]

# verify row sums approx 1 for an example
example = master_df.iloc[0]
print('Age sum:', example[[c for c in master_df.columns if c.startswith('age_')]].sum())
print('Income sum:', example[[c for c in master_df.columns if c.startswith('income_')]].sum())

Age sum: 1.0
Income sum: 1.0


In [7]:
# 7. example: create simple weighted demographic index
# (for demonstration; not necessarily meaningful)

# assign numeric proxies to age groups
age_scores = {'<18': 1, '18-34': 2, '35-54': 3, '55+': 4}
master_df['age_index'] = sum(
    master_df[f'age_{ag}'] * score for ag, score in age_scores.items()
)

# assign numeric proxies to income levels
income_scores = {'Low': 1, 'Medium': 2, 'High': 3}
master_df['income_index'] = sum(
    master_df[f'income_{inc}'] * score for inc, score in income_scores.items()
)

master_df[['age_index', 'income_index']].head()

,age_index,income_index
0,2.73,2.15
1,1.75,2.09
2,2.06,2.15
3,2.36,2.20
4,2.50,1.43


In [8]:
# 8. merge store metadata
data_full = master_df.merge(stores_df, on='store_id', how='left')

# example preview
data_full.head()

# export to CSV
output_dir = Path('data')
output_dir.mkdir(parents=True, exist_ok=True)
csv_path = output_dir / 'van_leeuwen_synthetic_sales.csv'
data_full.to_csv(csv_path, index=False)

print(f'Synthetic dataset saved to {csv_path}')

Synthetic dataset saved to data/van_leeuwen_synthetic_sales.csv


In [9]:
# total sales per quarter
sales_q = data_full.groupby('quarter')['sales'].sum().sort_index()
sales_q

quarter
2019Q1   52,625,747.07
2019Q2   53,154,223.91
2019Q3   52,776,968.71
2019Q4   51,969,643.78
2020Q1   52,512,361.29
2020Q2   53,195,542.79
2020Q3   53,307,810.06
2020Q4   52,550,626.28
2021Q1   51,114,879.72
2021Q2   52,866,073.92
2021Q3   52,160,831.95
2021Q4   52,570,824.62
2022Q1   51,324,626.40
2022Q2   52,435,382.65
2022Q3   52,269,008.06
2022Q4   52,848,822.65
2023Q1   53,105,205.98
2023Q2   52,548,020.21
2023Q3   52,426,854.36
2023Q4   51,926,729.83
2024Q1   52,153,487.08
2024Q2   51,922,213.31
2024Q3   51,402,392.43
2024Q4   51,439,468.17
Name: sales, dtype: float64

In [10]:
# sales by channel for the most recent quarter
recent_q = data_full['quarter'].max()
channel_sales = data_full[data_full['quarter'] == recent_q].groupby('channel')['sales'].sum()
channel_sales

channel
Online Delivery    7,494,628.40
Retail Grocery    14,018,388.36
Scoop Shop        18,469,077.81
Wholesale         11,457,373.60
Name: sales, dtype: float64

In [11]:
income_by_country = data_full.groupby('country')['income_index'].mean()
income_by_country

country
Australia   2.01
Canada      2.00
Germany     2.01
Japan       2.00
UK          2.00
USA         2.00
Name: income_index, dtype: float64